In [6]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(temperature=0.1)

In [7]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in {language}."), # AI의 기본 설정
    AIMessage(content="Ciao, mi chiamo {name}!"), # 가상 대화
    HumanMessage(content="What is the distance between {country_a} and {country_b}. Also, what is your name?"),
]

chat.invoke(messages)

AIMessage(content='La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Come posso aiutarti oggi?')